# ランダムフォレストによる分類

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# matplotlib: 日本語フォントの設定
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 
                               'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

# Iris データセット
iris = sns.load_dataset('iris')

# species が setosa のデータを除去
df = iris.query('species!="setosa"')
# speciesを 0, 1 にするためにダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# データセットの確認
df.iloc[[0, 50]]

,sepal_length,sepal_width,petal_length,petal_width,species_virginica
50,7.0,3.2,4.7,1.4,0
100,6.3,3.3,6.0,2.5,1


<hr>

### ランダムフォレストの精度の比較 (1)

説明変数

1. 一番重なりが少なそうな petal_width だけ
2. かなりの重なりがある petal_width だけ
3. 全部 (sepal_length, sepal_width, petal_length, petal_width)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier

# 説明変数のリストと目的変数の設定
X_labels_list = [
    ['petal_width'],
    ['sepal_width'],
    ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
]
Y = df.species_virginica

# ランダムフォレスト
model = RandomForestClassifier()

for i in range(0, len(X_labels_list)):
    # ラベルから説明変数の設定
    X_labels = X_labels_list[i]
    X = df[X_labels]
    # 学習
    model.fit(X, Y)
    # 教師データについてモデルからの予測値を計算
    Y_predict = model.predict(X)

    # 特異度の計算
    matrix = confusion_matrix(Y, Y_predict)
    specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])
    # 精度
    print('{}: {}'.format(i, X_labels))
    print('正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
        accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
        recall_score(Y, Y_predict), specificity))
    # ROC, AUC
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    plt.plot(fpr, tpr, label='{}: AUC={:.3f}'.format(i, auc(fpr, tpr)))

plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()

全部 (sepal_length, sepal_width, petal_length, petal_width)を説明変数とした場合の情報利得の総和

In [ ]:
pd.DataFrame(model.feature_importances_, index=X.columns, columns=['gain'])

### ランダムフォレストの精度の比較 (2)

- 説明変数は情報利得が大きい petal_length, petal_width を選択。
- 決定木の深さを変更。

In [ ]:
# 説明変数と目的変数の設定
X = df[['petal_length', 'petal_width']]
Y = df.species_virginica


for i in range(1, 6):
    # ランダムフォレスト
    model = RandomForestClassifier(max_depth=i)
    # 学習
    model.fit(X, Y)
    # 教師データについてモデルからの予測値を計算
    Y_predict = model.predict(X)

    # 特異度の計算
    matrix = confusion_matrix(Y, Y_predict)
    specificity = matrix[0, 0] / (matrix[0, 1] + matrix[0, 0])
    # 精度
    print('{}: 正確度: {:.3f}, 適合度: {:.3f}, 再現率: {:.3f}, 特異度: {:.3f}'.format(
        i, accuracy_score(Y, Y_predict), precision_score(Y, Y_predict),
        recall_score(Y, Y_predict), specificity))
    # ROC, AUC
    Y_proba = model.predict_proba(X)
    fpr, tpr, thresholds = roc_curve(Y, Y_proba[:, 1])
    plt.plot(fpr, tpr, label='{}: AUC={:.3f}'.format(i, auc(fpr, tpr)))

plt.xlabel('偽陽性率 (FP率)')
plt.ylabel('真陽性率 (TP率)')
plt.title('ROC曲線')
plt.legend()
plt.show()